## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-23-14-52-27 +0000,startribune,"Students honor Charlie Kirk at the U, one of T...",https://www.startribune.com/students-honor-cha...
1,2025-09-23-14-52-19 +0000,nyt,Live Updates: Trump Attacks U.N. and Lectures ...,https://www.nytimes.com/live/2025/09/23/world/...
2,2025-09-23-14-45-01 +0000,nypost,D4vd released creepy music video dragging body...,https://nypost.com/2025/09/23/us-news/d4vd-mad...
3,2025-09-23-14-43-31 +0000,nyt,‘Autism Doesn’t Need a Cure’: Trump’s Message ...,https://www.nytimes.com/2025/09/23/us/trump-au...
4,2025-09-23-14-40-25 +0000,bbc,Tories call for inquiry into Labour Together s...,https://www.bbc.com/news/articles/c79v74gy38lo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2365/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
14,trump,68
114,kimmel,19
3,kirk,17
113,jimmy,15
428,palestinian,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
155,2025-09-23-03-02-00 +0000,wsj,In labeling Tylenol use in pregnant women as a...,https://www.wsj.com/politics/in-targeting-a-co...,139
268,2025-09-22-20-35-04 +0000,nyt,Jimmy Kimmel’s Suspension Showed How Trump Ech...,https://www.nytimes.com/2025/09/22/business/ch...,123
242,2025-09-22-21-34-40 +0000,nypost,Trump announces FDA will change Tylenol safety...,https://nypost.com/2025/09/22/us-news/trump-an...,120
280,2025-09-22-19-59-40 +0000,nypost,Trump open to national holiday honoring Charli...,https://nypost.com/2025/09/22/us-news/trump-li...,116
173,2025-09-23-01-00-00 +0000,wsj,The worker visas that President Trump says now...,https://www.wsj.com/economy/jobs/h1b-visas-wor...,113


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
155,139,2025-09-23-03-02-00 +0000,wsj,In labeling Tylenol use in pregnant women as a...,https://www.wsj.com/politics/in-targeting-a-co...
283,88,2025-09-22-19-43-36 +0000,nypost,Jimmy Kimmel set for return to ABC after backl...,https://nypost.com/2025/09/22/business/jimmy-k...
16,51,2025-09-23-14-17-00 +0000,wsj,The Secret Service said it dismantled a networ...,https://www.wsj.com/politics/national-security...
235,41,2025-09-22-21-47-05 +0000,nyt,U.S. allies’ recognition of a Palestinian stat...,https://www.nytimes.com/live/2025/09/22/world/...
94,41,2025-09-23-10-00-00 +0000,nypost,New bill would crack down on bureaucrats worki...,https://nypost.com/2025/09/23/us-news/new-bill...
60,35,2025-09-23-12-00-00 +0000,nypost,Our baby was totally healthy until 8 months — ...,https://nypost.com/2025/09/23/health/hope-on-t...
332,35,2025-09-22-16-36-33 +0000,nypost,‘The View’ co-hosts finally break silence on J...,https://nypost.com/2025/09/22/media/the-view-c...
40,34,2025-09-23-13-22-01 +0000,nypost,France’s Emmanuel Macron calls Trump for help ...,https://nypost.com/2025/09/23/us-news/frances-...
288,32,2025-09-22-19-31-54 +0000,nypost,Hamas terrorists murder 3 Palestinians in Gaza...,https://nypost.com/2025/09/22/world-news/hamas...
173,31,2025-09-23-01-00-00 +0000,wsj,The worker visas that President Trump says now...,https://www.wsj.com/economy/jobs/h1b-visas-wor...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
